# Data Import & Connection to BigQuery

In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from google.cloud import bigquery
# set display options to show all columns
pd.set_option('display.max_columns', None)
# Set the float format to display numbers without scientific notation
pd.options.display.float_format = '{:.2f}'.format
# Set the client for future queries to BigQuery
client = bigquery.Client(project = "continente-lced-feup")

c:\Users\ian.karkles\Anaconda3\lib\site-packages\google\auth\_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [54]:
!gcloud auth application-default login

Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=AqG13ND1GWOmdumdQO72CvTY3HsyzV&access_type=offline&code_challenge=vtc0g8V1BzH04A1a5-nOEuLN4d6VnQDrSgOt-ZDYwsE&code_challenge_method=S256


Credentials saved to file: [C:\Users\ian.karkles\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot add the 

# Data Loading

In [55]:
query = client.query("""
   SELECT *
   FROM 
       tables_raw.fact_transaction 
       JOIN tables_raw.dim_customer USING(CUSTOMER_ACCOUNT_NR_MASK)
       JOIN tables_raw.dim_location USING(LOCATION_CD)
       JOIN tables_raw.dim_product USING(SKU)
   WHERE
       SEG_AGE_DSC="]25;35]"
   LIMIT 100000
   """)

df = query.result().to_dataframe() # Wait for the job to complete.
df

,SKU,LOCATION_CD,CUSTOMER_ACCOUNT_NR_MASK,TIME_KEY,TRANSACTION_ID_MASK,LOC_BRAND_CD,POS_TP_CD,PRODUCT_KEY,QTY,NET_SLS_AMT,GROSS_SLS_AMT,PROD_DSCNT_ISSUED_AMT,TRANS_DSCNT_RAT_AMT,DIRECT_DSCNT_AMT,GENDER,FAMILY_MEMBERS,CP4,seg_lifestyle_cd,seg_lifestyle_dsc,SEG_AGE,SEG_AGE_DSC,seg_lifestage_cd,seg_lifestage_dsc,LOCATION_DSC,LOC_BRAND_CD_1,LOC_BRAND_DSC,cp7,PRODUCT_DSC,UNIT_BASE_CD_EXT,UNIT_BASE_DSC_EXT,SUBCAT_CD_EXT,SUBCAT_DSC_EXT,CAT_CD_EXT,CAT_DSC_EXT,BIZ_UNIT_CD_EXT,BIZ_UNIT_DSC_EXT,DEPARTMENT_CD_EXT,DEPARTMENT_DSC_EXT,PRODUCT_SHORT_DSC,BRAND_DSC,BRAND_TYPE_CD,PRICE_RANGE,CONVERSION_FACTOR,CAPACITY_UNIT
0,6950077,3466,113317692.00,20221030,8787806088922310251,302,P,6950077010001,-1.00,-2.50,-2.50,0.00,0.00,0.00,None,<NA>,None,3,Preço,SA_3,]25;35],3,Family w/ Young Adul,CBD SOBREDA,302,continente bom dia,2815,ESCORREDOR LEGUMES C/ CABO CZ ESSENCIAL,37010301,37010301 - acess coz plástico,370103,370103 - acessórios cozinha,3701,3701 - cozinha,37,37 - Casa-Cozinha/Lavand,21,21 - CASA,ESCORREDOR LEGU,KASA,MP,SECUNDARIA,1.00,UN
1,7104290,325,87555786.00,20210131,6962053455751522303,302,P,7104290010001,-1.00,-2.50,-2.50,0.00,0.00,0.00,F,1,4415,1,Qualidade,SA_3,]25;35],5,Active Adults,CBD BOUCINHAS,302,continente bom dia,4430,CJ. 50 VELAS TEALIGHT BRANCAS,40020402,40020402 - tealights,400204,400204 - velas,4002,4002 - decora�ão,40,40 - Casa-Conforto,21,21 - CASA,CJ.50 TEALIGHTS,KASA,MP,ECONOMICA,50.00,UN
2,4866687,3193,59791069.00,20210509,7384098880584488637,302,P,4866687010001,1.00,3.00,3.69,0.00,0.37,0.00,F,3,3420,2,Família,SA_3,]25;35],5,Active Adults,CBD OLIVEIRA HOSPITA,302,continente bom dia,3400,BANDAS PONTOS NEGROS MYLABEL 8UNID,9030104,09030104 - Tratamento Acne,90301,090301 - rosto,903,0903 - perfum. e cosmética,9,09 - Beleza,10,10 - ALIMENTAR,BANDAS MYLABEL,MYLABEL,MP,PROPRIA,8.00,UN
3,6437059,1706,83498035.00,20210502,7377308729725263103,302,P,6437059010001,2.00,3.00,3.18,0.00,0.00,0.00,F,3,2775,1,Qualidade,SA_3,]25;35],3,Family w/ Young Adul,CBD DEFENSORES CHAVE,302,continente bom dia,1000,BEBIDA DE AMÊNDOA CONTINENTE BIO 1L,8050250,08050250 - Beb Veg Frtsec bio,80502,080502 - Beb Vegetal Frt seco,805,0805 - Bebida&Creme Vegetal,8,08 - Laticínios/Beb. Veg.,10,10 - ALIMENTAR,BEBIDA AMÊNDOA,CONTINENTE BIO,MP,SECUNDARIA,1.00,LT
4,4335919,2754,83520001.00,20220320,8852242922222659273,302,P,4335919010001,1.00,3.00,3.69,0.00,0.00,0.00,F,<NA>,3025,2,Família,SA_3,]25;35],4,Family with Kids,CBD COIMBRA AUTOIND,302,continente bom dia,3040,LAM. DESC.GILLETTE BLUE II PLUS 5+1,9010201,09010201 - Até 2 Lâminas,90102,090102 - máquinas descart.,901,0901 - produtos para homem,9,09 - Beleza,10,10 - ALIMENTAR,LAM.DES.GIL.5+1,GILLETTE,MF,SECUNDARIA,5.00,UN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,4850075,6332,104573031.00,20211005,7210047598652123838,303,P,4850075010001,1.00,7.06,7.48,0.00,0.78,3.21,M,3,2480,2,Família,SA_3,]25;35],4,Family with Kids,MDL PORTO DE MOS,303,continente modelo,2480,TOALHITAS DODOT SENSITIVE 6X54UN,5040201,05040201 - toalhitas bebé,50402,050402 - toalhitas bebé&crian,504,0504 - hig e prote�ão bebé,5,05 - Higiene,10,10 - ALIMENTAR,T DOD SENS 6X54,DODOT,MF,LIDER,1.00,UN
99996,6395692,843,40308501.00,20210919,7229988268770757973,303,P,6395692010001,1.00,1.23,1.29,0.00,0.00,0.00,F,3,9125,1,Qualidade,SA_3,]25;35],4,Family with Kids,MDL RIB. BRAVA,303,continente modelo,9350,TOALHITAS HIDRATA�ÃO CNT DO BEBÉ 72UN,5040201,05040201 - toalhitas bebé,50402,050402 - toalhitas bebé&crian,504,0504 - hig e prote�ão bebé,5,05 - Higiene,10,10 - ALIMENTAR,TOAL BEBÉ 72UN,CONT.DO BEBÉ,MP,PROPRIA,72.00,UN
99997,6840660,250,87618026.00,20210806,7438336631130301211,303,P,6840660010001,1.00,2.90,3.57,0.00,0.00,1.92,M,3,4420,3,Preço,SA_3,]25;35],5,Active Adults,MDL S. COSME,303,continente modelo,4420,CHAMPÔ J'S BABY GOTAS DE BRILHO 500ML,5040402,05040402 - Cabelo Criança,50404,050404 - perfumaria

# Data Preparation (more to be done...)

In [56]:
df = df.drop(columns=['LOC_BRAND_CD','PROD_DSCNT_ISSUED_AMT','NET_SLS_AMT','TRANS_DSCNT_RAT_AMT','DIRECT_DSCNT_AMT','CP4',
                 'seg_lifestyle_dsc','SEG_AGE','SEG_AGE_DSC','seg_lifestage_dsc','LOCATION_DSC','LOC_BRAND_DSC','cp7',
                 'UNIT_BASE_DSC_EXT','SUBCAT_CD_EXT','SUBCAT_DSC_EXT','BIZ_UNIT_DSC_EXT','DEPARTMENT_DSC_EXT',
                'PRODUCT_SHORT_DSC','BRAND_DSC','BRAND_TYPE_CD','CONVERSION_FACTOR','CAPACITY_UNIT'])

df  # Remover devoluções?

,SKU,LOCATION_CD,CUSTOMER_ACCOUNT_NR_MASK,TIME_KEY,TRANSACTION_ID_MASK,POS_TP_CD,PRODUCT_KEY,QTY,GROSS_SLS_AMT,GENDER,FAMILY_MEMBERS,seg_lifestyle_cd,seg_lifestage_cd,LOC_BRAND_CD_1,PRODUCT_DSC,UNIT_BASE_CD_EXT,CAT_CD_EXT,CAT_DSC_EXT,BIZ_UNIT_CD_EXT,DEPARTMENT_CD_EXT,PRICE_RANGE
0,6950077,3466,113317692.00,20221030,8787806088922310251,P,6950077010001,-1.00,-2.50,None,<NA>,3,3,302,ESCORREDOR LEGUMES C/ CABO CZ ESSENCIAL,37010301,3701,3701 - cozinha,37,21,SECUNDARIA
1,7104290,325,87555786.00,20210131,6962053455751522303,P,7104290010001,-1.00,-2.50,F,1,1,5,302,CJ. 50 VELAS TEALIGHT BRANCAS,40020402,4002,4002 - decora�ão,40,21,ECONOMICA
2,4866687,3193,59791069.00,20210509,7384098880584488637,P,4866687010001,1.00,3.69,F,3,2,5,302,BANDAS PONTOS NEGROS MYLABEL 8UNID,9030104,903,0903 - perfum. e cosmética,9,10,PROPRIA
3,6437059,1706,83498035.00,20210502,7377308729725263103,P,6437059010001,2.00,3.18,F,3,1,3,302,BEBIDA DE AMÊNDOA CONTINENTE BIO 1L,8050250,805,0805 - Bebida&Creme Vegetal,8,10,SECUNDARIA
4,4335919,2754,83520001.00,20220320,8852242922222659273,P,4335919010001,1.00,3.69,F,<NA>,2,4,302,LAM. DESC.GILLETTE BLUE II PLUS 5+1,9010201,901,0901 - produtos para homem,9,10,SECUNDARIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,4850075,6332,104573031.00,20211005,7210047598652123838,P,4850075010001,1.00,7.48,M,3,2,4,303,TOALHITAS DODOT SENSITIVE 6X54UN,5040201,504,0504 - hig e prote�ão bebé,5,10,LIDER
99996,6395692,843,40308501.00,20210919,7229988268770757973,P,6395692010001,1.00,1.29,F,3,1,4,303,TOALHITAS HIDRATA�ÃO CNT DO BEBÉ 72UN,5040201,504,0504 - hig e prote�ão bebé,5,10,PROPRIA
99997,6840660,250,87618026.00,20210806,7438336631130301211,P,6840660010001,1.00,3.57,M,3,3,5,303,CHAMPÔ J'S BABY GOTAS DE BRILHO 500ML,5040402,504,0504 - hig e prote�ão bebé,5,10,SECUNDARIA
99998,2055099,845,4372692.00,20210125,6955829703193656963,P,2055099010001,1.00,2.99,F,4,2,4,303,COTONETES JOHNSON'S 200UN,5040305,504,0504 - hig e prote�ão bebé,5,10,SECUNDARIA


In [57]:
df.isnull().sum()  # Maybe we shouldn't remove null values... What they mean? Insert values so that we don't lose precious data

SKU                             0
LOCATION_CD                     0
CUSTOMER_ACCOUNT_NR_MASK        0
TIME_KEY                        0
TRANSACTION_ID_MASK             0
POS_TP_CD                       0
PRODUCT_KEY                     0
QTY                             0
GROSS_SLS_AMT                   0
GENDER                       4913
FAMILY_MEMBERS              18812
seg_lifestyle_cd                0
seg_lifestage_cd                0
LOC_BRAND_CD_1                  0
PRODUCT_DSC                     0
UNIT_BASE_CD_EXT                0
CAT_CD_EXT                      0
CAT_DSC_EXT                     0
BIZ_UNIT_CD_EXT                 0
DEPARTMENT_CD_EXT               0
PRICE_RANGE                     0
dtype: int64

# Feature Engineering

Explicit Features Engineering

In [58]:
# convert the 'TIME_KEY' column to datetime format
df['TIME_KEY'] = pd.to_datetime(df['TIME_KEY'], format='%Y%m%d')

# create new columns for the day, week, day of the week, month, quarter, and year
df['DAY'] = df['TIME_KEY'].dt.day
df['WEEK'] = df['TIME_KEY'].dt.week
df['DOW'] = df['TIME_KEY'].dt.dayofweek
df['MONTH'] = df['TIME_KEY'].dt.month
df['QUARTER'] = df['TIME_KEY'].dt.quarter
df['YEAR'] = df['TIME_KEY'].dt.year

C:\Users\ian.karkles\AppData\Local\Temp\ipykernel_19428\715188214.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['WEEK'] = df['TIME_KEY'].dt.week


Customer Feature Engineering

In [59]:
# TOTAL NUMBER OF ITEMS BOUGHT

# sort the data by customer account and date
df = df.sort_values(['CUSTOMER_ACCOUNT_NR_MASK', 'TIME_KEY'])

# group the data by customer account and count the number of transactions up to that date
df['CUST_NUM_ITEMS_BOUGHT'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK')['TIME_KEY'].cumcount()+1

In [60]:
# TOTAL NUMBER OF PRIOR ORDERS

def count_unique_transactions(customer_transactions):
    prev_transaction = None
    count = -1
    counts = []
    for transaction in customer_transactions:
        if transaction != prev_transaction:
            count += 1
        counts.append(count)
        prev_transaction = transaction
    return counts

df['CUST_NUM_PRIOR_TRANSACTIONS'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK')['TRANSACTION_ID_MASK'].transform(count_unique_transactions)
df['CUST_NUM_PRIOR_TRANSACTIONS'] = df['CUST_NUM_PRIOR_TRANSACTIONS'].astype(int)

In [61]:
# UNIQUE NUMBER OF CATEGORIES BOUGHT

def count_unique(customer_categories):
    prev_categories = []
    count = 0
    counts = []
    for category in customer_categories:
        if category not in prev_categories:
            count += 1
        counts.append(count)
        prev_categories.append(category)
    return counts

df['CUST_NUM_UNIQUE_CATEGORIES'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK')['CAT_CD_EXT'].transform(count_unique)
df['CUST_NUM_UNIQUE_CATEGORIES'] = df['CUST_NUM_UNIQUE_CATEGORIES'].astype(int)

In [62]:
# UNIQUE NUMBER OF ORDERS??? Makes sense at all?

In [63]:
# DAYS SINCE LAST TRANSACTION

df['LAST_TIME_KEY'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK')['TIME_KEY'].shift()
df['CUST_DAYS_SINCE_LAST_TRANSACTION'] = (df['TIME_KEY'] - df['LAST_TIME_KEY']).dt.days.fillna(-1)
df['CUST_DAYS_SINCE_LAST_TRANSACTION'] = df['CUST_DAYS_SINCE_LAST_TRANSACTION'].astype(int)
df = df.drop(columns=['LAST_TIME_KEY'])

In [64]:
# AVERAGE DAYS SINCE LAST TRANSACTION

def calculate_rolling_avg(df):
    customer_groups = df.groupby('CUSTOMER_ACCOUNT_NR_MASK')
    for name, group in customer_groups:
        # Calculate the rolling average for each customer
        rolling_avg = group['CUST_DAYS_SINCE_LAST_TRANSACTION'].rolling(window=1, min_periods=1).mean()
        # Reset the rolling average to 0 for the first transaction of each customer
        rolling_avg.iloc[0] = 0
        # Set the rolling average values back into the original dataframe
        df.loc[group.index, 'rolling_avg_days_since_prior_transaction'] = rolling_avg.values
    # Calculate the sum and count of the rolling average for each customer
    df['rolling_avg_days_since_prior_transaction_sum'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK')['rolling_avg_days_since_prior_transaction'].cumsum()
    # Calculate the customer average for the rolling average
    df['CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION'] = df['rolling_avg_days_since_prior_transaction_sum'] / df['CUST_NUM_PRIOR_TRANSACTIONS']
    # Round the customer average to 2 decimal places
    df['CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION'] = df['CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION'].round(2).fillna(0)
    df = df.drop(columns=['rolling_avg_days_since_prior_transaction','rolling_avg_days_since_prior_transaction_sum'])
    return df


df = calculate_rolling_avg(df)

In [65]:
# SIZE OF THE LAST BASKET

def add_cust_last_transaction_count(df):
    df = df.sort_values(['CUSTOMER_ACCOUNT_NR_MASK', 'TIME_KEY'])
    
    df['CUST_LAST_BASKET_SIZE'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK')['TRANSACTION_ID_MASK'].apply(lambda x: x.shift().map(x.value_counts()).fillna(0)).astype(int)
    
    # update values for rows with repeated transaction IDs
    for _, group in df.groupby('TRANSACTION_ID_MASK'):
        if len(group) > 1:
            first_row_index = group.index[0]
            df.loc[group.index, 'CUST_LAST_BASKET_SIZE'] = df.at[first_row_index, 'CUST_LAST_BASKET_SIZE'] 
    return df

df = add_cust_last_transaction_count(df)

C:\Users\ian.karkles\AppData\Local\Temp\ipykernel_19428\291114631.py:6: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df['CUST_LAST_BASKET_SIZE'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK')['TRANSACTION_ID_MASK'].apply(lambda x: x.shift().map(x.value_counts()).fillna(0)).astype(int)


In [66]:
# AVERAGE BASKET SIZE - V2 -Ian's Fix
# Remove transactions with the same negative quantity
def calculate_avg_basket(df):
    df_avg_bkt_size = df[df['QTY'] > 0]
    df_avg_bkt_size = df[['CUSTOMER_ACCOUNT_NR_MASK','TRANSACTION_ID_MASK','CAT_DSC_EXT']]
    df_avg_bkt_size = df_avg_bkt_size.groupby(['CUSTOMER_ACCOUNT_NR_MASK','TRANSACTION_ID_MASK']).count()
    df_avg_bkt_size = df_avg_bkt_size.groupby('CUSTOMER_ACCOUNT_NR_MASK').mean().round(0).astype(int)
    df_avg_bkt_size.reset_index(inplace=True)
    df_avg_bkt_size = df_avg_bkt_size.rename(columns={'CAT_DSC_EXT':'CUST_AVG_BASKET_SIZE'})
    df = df.merge(df_avg_bkt_size, on='CUSTOMER_ACCOUNT_NR_MASK', how='left')
    return df

df = calculate_avg_basket(df)

In [67]:
df

,SKU,LOCATION_CD,CUSTOMER_ACCOUNT_NR_MASK,TIME_KEY,TRANSACTION_ID_MASK,POS_TP_CD,PRODUCT_KEY,QTY,GROSS_SLS_AMT,GENDER,FAMILY_MEMBERS,seg_lifestyle_cd,seg_lifestage_cd,LOC_BRAND_CD_1,PRODUCT_DSC,UNIT_BASE_CD_EXT,CAT_CD_EXT,CAT_DSC_EXT,BIZ_UNIT_CD_EXT,DEPARTMENT_CD_EXT,PRICE_RANGE,DAY,WEEK,DOW,MONTH,QUARTER,YEAR,CUST_NUM_ITEMS_BOUGHT,CUST_NUM_PRIOR_TRANSACTIONS,CUST_NUM_UNIQUE_CATEGORIES,CUST_DAYS_SINCE_LAST_TRANSACTION,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION,CUST_LAST_BASKET_SIZE,CUST_AVG_BASKET_SIZE
0,5208104,222,31655.00,2021-04-23,7368257087394012105,P,5208104010001,1.00,1.79,F,3,2,4,303,PÃO DE SEMENTES E LINHAÇA OROWEAT 680G,16030105,1603,1603 - pão industrial&tosta,16,12,SECUNDARIA,23,16,4,4,2,2021,1,0,1,-1,0.00,0,1
1,5619216,222,31655.00,2021-04-23,7368257087394012105,P,5619216010001,2.00,1.78,F,3,2,4,303,MINI FOFOS 300 GR CONTINENTE,16030303,1603,1603 - pão industrial&tosta,16,12,SECUNDARIA,23,16,4,4,2,2021,2,0,1,0,0.00,0,1
2,2237563,222,31655.00,2022-03-23,8856282000637385347,P,2237563010001,1.00,2.09,F,3,2,4,303,VQR PALITOS 140GR,13010603,1301,1301 - queijos,13,12,SECUNDARIA,23,12,2,3,1,2022,3,1,2,334,334.00,2,1
3,2163597,222,31655.00,2022-10-10,8731843168703891348,P,2163597010001,1.00,2.29,F,3,2,4,303,QJ MINI BABYBEL 6*20G,13010603,1301,1301 - queijos,13,12,SECUNDARIA,10,41,0,10,4,2022,4,2,2,201,267.50,1,1
4,2258396,222,31655.00,2022-12-16,9088179212458285683,P,2258396010001,1.00,2.39,F,3,2,4,303,PASTA AQUAFRESH T.PROT.MENTA FRESC 75 ML,5010101,501,0501 - higiene oral,5,10,SECUNDARIA,16,50,4,12,4,2022,5,3,3,67,200.67,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2588815,927,133321689.00,2022-12-28,9136135750280186694,P,2588815010001,0.74,5.21,None,<NA>,4,6,143,PORCO LOMBINHOS VÁCUO - LS,11020206,1102,1102 - suíno,11,11,PROPRIA,28,52,2,12,4,2022,1,0,1,-1,0.00,0,1
99996,7402211,261,133329843.00,2022-12-13,9084144537457409850,P,7402211010001,1.00,3.99,F,2,4,6,303,ÁGUA DE COCO COCOXIM 1L,3010301,301,0301 - sumos e néctares,3,10,SECUNDARIA,13,50,1,12,4,2022,1,0,1,-1,0.00,0,2
99997,2076764,261,133329843.00,2022-12-13,9084144537457409850,P,2076764010001,0.47,1.19,F,2,4,6,303,COUVE BROCOLO CNT KG,15020402,1502,1502 - legumes,15,12,PROPRIA,13,50,1,12,4,2022,2,0,2,0,0.00,0,2
99998,7275586,261,133329843.00,2022-12-27,9135163101682601903,P,7275586010001,2.00,1.48,F,2,4,6,303,SAQ.FRUTA FRUTOS TROPICAIS CONTINENTE 90,14020402,1402,1402 - alimenta�ão infantil,14,10,SECUNDARIA,27,52,1,12,4,2022,3,1,3,14,14.00,2,2


Category features

In [68]:
# TOTAL NUMBER OF CATEGORIES BOUGHT

df = df.sort_values('TIME_KEY')
df['CAT_NUM_CAT_BOUGHT'] = df.groupby('CAT_DSC_EXT')['TIME_KEY'].rank(method='first')
df['CAT_NUM_CAT_BOUGHT'] = df['CAT_NUM_CAT_BOUGHT'].astype(int)
df = df.sample(frac = 1)

In [69]:
# NUMBER OF CUSTOMERS WHO BOUGHT A CATEGORY

df.sort_values(['CUSTOMER_ACCOUNT_NR_MASK', 'CAT_DSC_EXT', 'TIME_KEY'], inplace=True)
first_appearance_mask = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'CAT_DSC_EXT'])['TIME_KEY'].transform(lambda x: x == x.min())
df['CAT_CUST_BOUGHT_CAT'] = first_appearance_mask.groupby(df['CAT_DSC_EXT']).cumsum()
df['CAT_CUST_BOUGHT_CAT'].fillna(0, inplace=True)
df['CAT_CUST_BOUGHT_CAT'] = df['CAT_CUST_BOUGHT_CAT'].astype(int)

Customer-category features

In [70]:
# TOTAL NUMBER OF REORDERS PER CUSTOMER

df['CAT_REORDER_NUM'] = 0
for customer, category in df[['CUSTOMER_ACCOUNT_NR_MASK', 'CAT_DSC_EXT']].drop_duplicates().values:
    mask = (df['CUSTOMER_ACCOUNT_NR_MASK'] == customer) & (df['CAT_DSC_EXT'] == category)
    df.loc[mask, 'CAT_REORDER_NUM'] = df[mask]['TIME_KEY'].rank(method='min') - 1

In [71]:
# NUMBER OF CUSTOMERS WHO BOUGHT ONLY ONE TIME

counts = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'CAT_DSC_EXT']).size().to_dict()
def is_unique(row):
    if counts[(row['CUSTOMER_ACCOUNT_NR_MASK'], row['CAT_DSC_EXT'])] == 1:
        return 1
    else:
        return 0


df['CAT_CUS_ONLY_1T'] = df.apply(is_unique, axis=1)

In [72]:
# DAYS SINCE A CUSTOMER BOUGHT A CATEGORY
# DAYS SINCE THE FIRST CATEGORY WAS BOUGHT


df = df.sort_values(['CUSTOMER_ACCOUNT_NR_MASK', 'TIME_KEY'])
df['DAYS_SINCE_LAST_PURCHASE'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'CAT_DSC_EXT'])['TIME_KEY'].diff().dt.days.fillna(0)
df['DAYS_SINCE_FIRST_PURCHASE'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'CAT_DSC_EXT'])['DAYS_SINCE_LAST_PURCHASE'].cumsum()
df['DAYS_SINCE_FIRST_PURCHASE'] = df['DAYS_SINCE_FIRST_PURCHASE'].astype(int)

df['DAYS_SINCE_FIRST_PURCHASE'] = pd.to_timedelta(df['DAYS_SINCE_FIRST_PURCHASE'], unit='D')
df['DAYS_SINCE_LAST_PURCHASE'] = pd.to_timedelta(df['DAYS_SINCE_LAST_PURCHASE'], unit='D')

# df.drop('DAYS_SINCE_LAST_PURCHASE', axis=1, inplace=True)

df['DAYS_SINCE_LAST_PURCHASE'] = df['DAYS_SINCE_LAST_PURCHASE'].dt.floor('D')
df['DAYS_SINCE_FIRST_PURCHASE'] = df['DAYS_SINCE_FIRST_PURCHASE'].dt.floor('D')

In [73]:
#LABEL

df['LABEL'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'CAT_DSC_EXT'])['TIME_KEY'].apply(lambda x: ((x >= pd.Timestamp('2022-01-01')) & ((x - x.max()) <= pd.Timedelta(days=365))).astype(int))

C:\Users\ian.karkles\AppData\Local\Temp\ipykernel_19428\2824128452.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df['LABEL'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'CAT_DSC_EXT'])['TIME_KEY'].apply(lambda x: ((x >= pd.Timestamp('2022-01-01')) & ((x - x.max()) <= pd.Timedelta(days=365))).astype(int))


In [74]:
df = df.sample(frac = 1)
df.head(10)

,SKU,LOCATION_CD,CUSTOMER_ACCOUNT_NR_MASK,TIME_KEY,TRANSACTION_ID_MASK,POS_TP_CD,PRODUCT_KEY,QTY,GROSS_SLS_AMT,GENDER,FAMILY_MEMBERS,seg_lifestyle_cd,seg_lifestage_cd,LOC_BRAND_CD_1,PRODUCT_DSC,UNIT_BASE_CD_EXT,CAT_CD_EXT,CAT_DSC_EXT,BIZ_UNIT_CD_EXT,DEPARTMENT_CD_EXT,PRICE_RANGE,DAY,WEEK,DOW,MONTH,QUARTER,YEAR,CUST_NUM_ITEMS_BOUGHT,CUST_NUM_PRIOR_TRANSACTIONS,CUST_NUM_UNIQUE_CATEGORIES,CUST_DAYS_SINCE_LAST_TRANSACTION,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION,CUST_LAST_BASKET_SIZE,CUST_AVG_BASKET_SIZE,CAT_NUM_CAT_BOUGHT,CAT_CUST_BOUGHT_CAT,CAT_REORDER_NUM,CAT_CUS_ONLY_1T,DAYS_SINCE_LAST_PURCHASE,DAYS_SINCE_FIRST_PURCHASE,LABEL
91150,7473538,215,102861382.00,2022-01-23,7320163240166623192,P,7473538010001,1.00,1.79,F,3,1,5,143,IOG MAG C DANONE NATUR COCO 4*115G,8040603,804,0804 - iogurtes e sobrem.,8,10,SECUNDARIA,23,3,6,1,1,2022,18,12,12,14,21.58,1,1,2760,2759,1,0,14 days,14 days,1
83668,3872611,3194,87715510.00,2022-12-28,9136133435023531632,P,3872611010001,1.00,0.99,F,2,3,4,302,BOL.C/PEPITAS CHOCOLATE CONTINENTE 150G,2030205,203,0203 - bolachas,2,10,PROPRIA,28,52,2,12,4,2022,40,28,23,0,17.11,2,1,3439,1778,8,0,26 days,479 days,1
22293,7173279,3698,19090893.00,2022-09-15,8743013460133673207,P,7173279010001,1.00,1.09,M,2,2,4,302,BATATA FRITA OND KETCHUP CONTINENTE 170G,1010104,101,0101 - aperitivos,1,10,SECUNDARIA,15,37,3,9,3,2022,17,14,7,20,44.21,1,1,1993,343,0,1,0 days,0 days,1
53395,7589297,2403,58414300.00,2022-09-05,8696755861140737237,P,7589297010001,1.00,12.49,F,1,1,4,302,DET LÍQ MÁQ ROUPA SKIP BABY 60D,6010201,601,0601 - limp. e tratam roupa,6,10,SECUNDARIA,5,36,0,9,3,2022,50,33,28,0,16.70,1,1,880,519,0,1,0 days,0 days,1
70393,7279167,9665,80389522.00,2021-08-30,7462187326230047474,P,7279167010001,1.00,1.75,F,5,3,5,302,PACK CROIS FOLHADO 5 + 1 OFERTA(AB),16020603,1602,1602 - pastelaria,16,12,SECUNDARIA,30,35,0,8,3,2021,3,2,3,58,109.00,1,1,665,1051,0,0,0 days,0 days,0
29718,5519021,224,26376838.00,2021-05-01,7376301038959839255,P,5519021010001,1.00,2.99,F,2,1,4,303,MI.PIZZA PICCOLINIS FRANK BUITONI 270G,7020402,702,0702 - refei�ões congeladas,7,10,PREMIUM,1,17,5,5,2,2021,1,0,1,-1,0.00,0,1,211,279,0,1,0 days,0 days,0
93589,5117048,1393,103640765.00,2022-02-06,8811207680245893446,P,5117048010001,2.00,2.68,None,<NA>,1,5,303,PÃO HAMBÚRGUER PANRICO 220GR,16030106,1603,1603 - pão industrial&tosta,16,12,SECUNDARIA,6,5,6,2,1,2022,2,1,1,229,229.00,1,1,1034,1438,1,0,229 days,229 days,1
82935,6923901,233,87643671.00,2021-07-29,7429201930499276257,P,6923901010001,1.00,1.39,F,4,1,4,303,GELADO CLASSICONE CHOC CNT 6*120ML,7030103,703,0703 - sobremesas cong,7,10,PROPRIA,29,30,3,7,3,2021,12,10,10,7,13.50,1,1,231,598,1,0,25 days,25 days,0
7293,2007257,202,4900863.00,2022-12-14,9080135198437025241,P,2007257010001,1.00,5.69,F,0,3,3,143,BOMBONS GUYLIAN FRUTOS MAR 250G,2020105,202,0202 - doçaria,2,10,SECUNDARIA,14,50,2,12,4,2022,2,1,2,0,0.00,1,1,2411,137,0,1,0 days,0 days,1
64281,3835925,251,71586363.00,2022-11-29,9107149611361449925,P,3835925010001,1.00,2.99,F,4,3,4,303,FEIJÃO VERDE CNT EMB.500GR,15020605,1502,1502 - legumes,15,12,SECUNDARIA,29,48,1,11,4,2022,19,15,15,0,45.53,1,1,4949,1716,0,1,0 days,0 days,1


In [75]:
df

,SKU,LOCATION_CD,CUSTOMER_ACCOUNT_NR_MASK,TIME_KEY,TRANSACTION_ID_MASK,POS_TP_CD,PRODUCT_KEY,QTY,GROSS_SLS_AMT,GENDER,FAMILY_MEMBERS,seg_lifestyle_cd,seg_lifestage_cd,LOC_BRAND_CD_1,PRODUCT_DSC,UNIT_BASE_CD_EXT,CAT_CD_EXT,CAT_DSC_EXT,BIZ_UNIT_CD_EXT,DEPARTMENT_CD_EXT,PRICE_RANGE,DAY,WEEK,DOW,MONTH,QUARTER,YEAR,CUST_NUM_ITEMS_BOUGHT,CUST_NUM_PRIOR_TRANSACTIONS,CUST_NUM_UNIQUE_CATEGORIES,CUST_DAYS_SINCE_LAST_TRANSACTION,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION,CUST_LAST_BASKET_SIZE,CUST_AVG_BASKET_SIZE,CAT_NUM_CAT_BOUGHT,CAT_CUST_BOUGHT_CAT,CAT_REORDER_NUM,CAT_CUS_ONLY_1T,DAYS_SINCE_LAST_PURCHASE,DAYS_SINCE_FIRST_PURCHASE,LABEL
91150,7473538,215,102861382.00,2022-01-23,7320163240166623192,P,7473538010001,1.00,1.79,F,3,1,5,143,IOG MAG C DANONE NATUR COCO 4*115G,8040603,804,0804 - iogurtes e sobrem.,8,10,SECUNDARIA,23,3,6,1,1,2022,18,12,12,14,21.58,1,1,2760,2759,1,0,14 days,14 days,1
83668,3872611,3194,87715510.00,2022-12-28,9136133435023531632,P,3872611010001,1.00,0.99,F,2,3,4,302,BOL.C/PEPITAS CHOCOLATE CONTINENTE 150G,2030205,203,0203 - bolachas,2,10,PROPRIA,28,52,2,12,4,2022,40,28,23,0,17.11,2,1,3439,1778,8,0,26 days,479 days,1
22293,7173279,3698,19090893.00,2022-09-15,8743013460133673207,P,7173279010001,1.00,1.09,M,2,2,4,302,BATATA FRITA OND KETCHUP CONTINENTE 170G,1010104,101,0101 - aperitivos,1,10,SECUNDARIA,15,37,3,9,3,2022,17,14,7,20,44.21,1,1,1993,343,0,1,0 days,0 days,1
53395,7589297,2403,58414300.00,2022-09-05,8696755861140737237,P,7589297010001,1.00,12.49,F,1,1,4,302,DET LÍQ MÁQ ROUPA SKIP BABY 60D,6010201,601,0601 - limp. e tratam roupa,6,10,SECUNDARIA,5,36,0,9,3,2022,50,33,28,0,16.70,1,1,880,519,0,1,0 days,0 days,1
70393,7279167,9665,80389522.00,2021-08-30,7462187326230047474,P,7279167010001,1.00,1.75,F,5,3,5,302,PACK CROIS FOLHADO 5 + 1 OFERTA(AB),16020603,1602,1602 - pastelaria,16,12,SECUNDARIA,30,35,0,8,3,2021,3,2,3,58,109.00,1,1,665,1051,0,0,0 days,0 days,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94217,6458580,324,104629652.00,2021-12-02,7302885218764091002,P,6458580010001,2.00,1.38,F,<NA>,1,5,302,CROISSANT FOLHADO CREME OVO (AB),16020603,1602,1602 - pastelaria,16,12,SECUNDARIA,2,48,3,12,4,2021,2,1,2,6,6.00,1,1,932,1380,0,1,0 days,0 days,0
37802,6683336,3698,34965200.00,2022-01-10,7342979663741549570,P,6683336010001,1.00,2.69,F,3,2,4,302,CHOURIÇO DA GUARDA CONTINENTE 200G PM,13020902,1302,1302 - Charcutaria,13,12,SECUNDARIA,10,2,0,1,1,2022,25,24,17,2,14.88,1,1,1289,673,1,0,195 days,195 days,1
20972,6410386,1908,18937039.00,2022-11-23,9101294664818024232,P,6410386010001,1.00,1.29,F,3,2,5,302,CEBOLA P/ ASSAR CNT 500G,15020702,1502,1502 - legumes,15,12,SECUNDARIA,23,47,2,11,4,2022,31,25,19,38,18.24,1,1,4899,548,4,0,104 days,247 days,1
95287,2224229,340,108150126.00,2021-09-25,7236058670342128018,P,2224229010001,1.00,5.99,None,<NA>,1,3,302,PAPEL ALUMINIO VILEDA 25M,6040502,604,0604 - prod papel e consum.,6,10,LIDER,25,38,5,9,3,2021,3,2,3,11,26.50,1,1,739,1414,0,0,0 days,0 days,0


In [ ]:
df.